# About the competition

*This is slighly modified problem than the sentiment analysis. Here we have to extract the sentiment sentences which cause the specific nature of senytence i.e.** which line of the sentences implies whether the sentiment is positive or negative.** This seems a little more intresting problem than the classical problem of sentiment classification.*

Without further delay let's dive in completely.

* # Evaluation Metric

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

Well this metric is quite easy to understand from the code give above. If you don't get it below is one example for your refrence

A = {0,1,2,5,6}

B = {0,2,3,4,5,7,9}

Solution: 
                 
                 = J(A,B) = |A∩B| / |A∪B| 

                 = |{0,2,5}| / |{0,1,2,3,4,5,6,7,9}|
                 
                 = 3/9 = 0.33.

For more please refer to this :
https://www.statisticshowto.datasciencecentral.com/jaccard-index/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Let't get our guns ready 
***(Import Libraries)***

In [ ]:
import numpy as np 
import pandas as pd 

# text processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Reading all datasets available

We have total of three csv file : train , test and submission

In [ ]:
#Training data
df_train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
df_sub = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')


In [ ]:
df_train.head()

In [ ]:
df_test.head()

So we can see select_text column is missing in the testing dataset ! 

In [ ]:
print(df_train.shape,df_test.shape)

Our dataset is of nominal size for training 

# We gonna go all the way down,guys!!

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()


Its strange that out of 27k rows only **1 text and 1 selected_text** is empty. 

This make me curious. Let's dig a bit more

In [ ]:
null_columns=df_train.columns[df_train.isnull().any()]
# print all rows with atleast one null values
print(df_train[df_train.isnull().any(axis=1)][null_columns])

Well we both nan value is in same column. I think we have all right to dump it ;)

In [ ]:
df_train.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)


In [ ]:
# Moving on to test dataset.

In [ ]:
df_test.isna().sum()

In [ ]:
# Nothing to worry about. 

In [ ]:
# Contains positive tweets 
# Here we have a glimpse of positive cases
df_pos = df_train[df_train['sentiment']=='positive']
df_pos['text'].head()

In [ ]:
df_neg = df_train[df_train['sentiment']=='negative']
df_neg['text'].head()

In [ ]:
df_neu = df_train[df_train['sentiment']=='neutral']
df_neu['text'].head()

# Data distribution based on Sentiment Column

In [ ]:
# Now lets check out whether tha dataset is distributed equally or not

In [ ]:
df_train['sentiment'].value_counts()


So the dataset which we have got is uneven with more more neutral text than positive and negative.

In [ ]:
sns.barplot(df_train['sentiment'].value_counts().index,df_train['sentiment'].value_counts(),palette='rocket')

In [ ]:
# Lets check out for tes dataset what is the proportions

In [ ]:
df_test['sentiment'].value_counts()

In [ ]:
sns.barplot(df_test['sentiment'].value_counts().index,df_test['sentiment'].value_counts(),palette='rocket')

From my intution we can see the test dataset is in same proportion !!

# No preprocessing! Let's do this bare-handed

From what I have understand this problem the sentiment extraction does not need any text preprocessing as even the extracted sentence do have stopwords and all this!

This is need for something special!! And wkt i.e. BERT( for now ;) )

# BERT is HERE!!

I will be using pytorch and Hugging face transformer for this task. Its quite easy to implement and use.

##  Install huggingface transformers library 

Please be sure to keep internet on!

In [ ]:
import torch

## Load Fine-Tuned BERT-large


For Question Answering we use the BertForQuestionAnswering class from the transformers library.

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

## Demo shot : Just in case for understanding

## For Easy-Pissy reproducible code we can use this amazing function

In [ ]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    #print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return answer

In [ ]:
text ='Spent the entire morning in a meeting w/ a vendor, and my boss was not happy w/ them. Lots of fun.I had other plans for my morning'
question = 'What text is neutral?'

In [ ]:
ans = answer_question(question, text)
print(ans)

# Voila! This does amazingly well!!

## Since this is a pretrained model we don't need to train model on the text.So we will directly apply the dataset

This may seem bizarre to few people but it's kinda cool to try.

In [ ]:
# Initialise the text
df_train['Bert_answers'] = ''

In [ ]:
df_train.head()

### We will ask BERT three questions

In [ ]:
positive_question = 'What text is positive ?' # In case of positive sentiment
negative_question = 'What text is negative ?'  # In case of negative sentiment
neutral_question = 'What text is neutral?' # In case of neutral sentiment

In [ ]:
df_test.shape[0]

In [ ]:

df_test['selected_text'] = ''
i = 0
while(i!=df_test.shape[0]):
    if (df_test['sentiment'].iloc[i]== 'positive'):
        df_test['selected_text'].iloc[i] = answer_question(positive_question,df_test['text'][i])
    elif (df_test['sentiment'].iloc[i]== 'negative'):
        df_test['selected_text'].iloc[i] = answer_question(negative_question,df_test['text'][i])
    else :
        df_test['selected_text'].iloc[i] = answer_question(neutral_question,df_test['text'][i])
    print(df_test['selected_text'].iloc[i])
    print(i)
    i = i+1
    
     


In [ ]:
df_test.head()

In [ ]:
df_test = df_test.drop(['text','sentiment'],axis = 1)
df_test.head()

In [ ]:
df_test.set_index('textID',inplace = True)
df_test.head()

In [ ]:
df_test.to_csv('submission.csv')

## This was gonna take foreever to see.We need to amp up our cpu power which is rather impossible

In [ ]:
# So we try to make efficent code

In [ ]:
# def soc_iter(sentiment,text):
#     if (sentiment == 'positive'):
#         result = answer_question(positive_question,text)
#     elif (sentiment == 'negative'):
#         result = answer_question(negative_question,text)
#     else :
#         result = answer_question(neutral_question,text)
        
    

In [ ]:
# %%timeit
# df_test['selected_text'] = ''
# draw_series = []
# for index, row in df_test.iterrows():
#     draw_series.append(soc_iter(row['sentiment'],row['text']))
#     print(index)
    
# df_test['selected_text'] = draw_series

In [ ]:
# We can see this method even though 321 times faster but still is gonna take an hour to complete